In [112]:
# Import our dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import hvplot.pandas

In [113]:
# Import input data
df = pd.read_csv('charity_data.csv')
display(df.head())
display(df.dtypes)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [114]:
# check for missing data
for column in df:
    null_count = df[column].isnull().sum()
    print(f'{column} has {null_count} missing data' )

EIN has 0 missing data
NAME has 0 missing data
APPLICATION_TYPE has 0 missing data
AFFILIATION has 0 missing data
CLASSIFICATION has 0 missing data
USE_CASE has 0 missing data
ORGANIZATION has 0 missing data
STATUS has 0 missing data
INCOME_AMT has 0 missing data
SPECIAL_CONSIDERATIONS has 0 missing data
ASK_AMT has 0 missing data
IS_SUCCESSFUL has 0 missing data


In [115]:
# Drop dupes
df = df.drop_duplicates()
df = df.dropna()
df.describe()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


In [116]:
df = df.drop(columns=["NAME"])
df.head()

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [117]:
df_obj_filter = df.dtypes[df.dtypes == object].index.tolist()
display(df_obj_filter)

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [118]:
# Lets see if bucketing is necessary
df[df_obj_filter].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [119]:

classification_count = df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C6100        1
C1580        1
C1370        1
C2380        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [120]:
classification_count.hvplot.density()

:Distribution   [CLASSIFICATION]   (Density)

In [121]:
classification_count.hvplot.bar()

:Bars   [index]   (CLASSIFICATION)

In [122]:
# Determine which values to replace for bucketing 
replace_classification = list(classification_count[classification_count<150].index)

#we iterate over small_count_list and replace items in charity_df['CLASSIFICATION']
for small_class in replace_classification:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(small_class,'Other')

#display reorganized column
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [123]:
application_type_count = df['APPLICATION_TYPE'].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [124]:
# Determine values to replace for bucketing
replace_application = list(application_type_count[application_type_count < 150].index)

# Replace in DataFrame
for application in replace_application:
    df.APPLICATION_TYPE = df.APPLICATION_TYPE.replace(application,"Other")

df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [125]:
# encode all categorical variables with OneHotEncoders
enc = OneHotEncoder(sparse=False)

# rerun the category definer
cat = df.dtypes[df.dtypes=="object"].index.tolist()

# fit and then produce the encoder
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [126]:
# Merge the encoded columns
df=df.merge(encode_df,left_index=True, right_index=True)

# Drop columns not encoded
df = df.drop(cat,1)

df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [135]:
# Making a copy of DF
df_2 = df.copy()
df_2

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,996010315,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,996012607,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,996015768,1,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [137]:
# Remove status from features data
y = df_2['IS_SUCCESSFUL'].values
x = df_2.drop('IS_SUCCESSFUL',1).values

# Split Split train & test datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [144]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)


# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)


In [223]:
# Define deep neural network model
number_input_features = len(x_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

neuro_network = tf.keras.models.Sequential()

# First hidden layer
neuro_network.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
neuro_network.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
neuro_network.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
neuro_network.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 80)                3920      
_________________________________________________________________
dense_38 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 31        
Total params: 6,381
Trainable params: 6,381
Non-trainable params: 0
_________________________________________________________________


In [224]:
# Compile the Sequential model together and customize metrics
neuro_network.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = neuro_network.fit(x_train_scaled, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 119us/sample - loss: 0.5738 - accuracy: 0.7193
Epoch 2/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5531 - accuracy: 0.7294
Epoch 3/100
25724/25724 [==============================] - 2s 85us/sample - loss: 0.5505 - accuracy: 0.7307
Epoch 4/100
25724/25724 [==============================] - 2s 84us/sample - loss: 0.5477 - accuracy: 0.7304
Epoch 5/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5447 - accuracy: 0.7332
Epoch 6/100
25724/25724 [==============================] - 2s 92us/sample - loss: 0.5432 - accuracy: 0.7328
Epoch 7/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5421 - accuracy: 0.7361
Epoch 8/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5413 - accuracy: 0.7354
Epoch 9/100
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5400 - accuracy: 0.7362
Epo

25724/25724 [==============================] - 2s 86us/sample - loss: 0.5151 - accuracy: 0.7502
Epoch 76/100
25724/25724 [==============================] - 2s 83us/sample - loss: 0.5148 - accuracy: 0.7499
Epoch 77/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5141 - accuracy: 0.7508
Epoch 78/100
25724/25724 [==============================] - 2s 83us/sample - loss: 0.5138 - accuracy: 0.7500
Epoch 79/100
25724/25724 [==============================] - 2s 85us/sample - loss: 0.5137 - accuracy: 0.7500
Epoch 80/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5144 - accuracy: 0.7491
Epoch 81/100
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5129 - accuracy: 0.7491
Epoch 82/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5138 - accuracy: 0.7510
Epoch 83/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5134 - accuracy: 0.7509
Epoch 84/100
25724/25724 [=====

In [225]:
# Evaluate the model using the test data
model_loss, model_accuracy = neuro_network.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5255 - accuracy: 0.7363
Loss: 0.5522030082284187, Accuracy: 0.7363265156745911
